# Spectroimaging pixel by pixel

Edited by Louise, 18/06/2020

Edited by Martin, 21/06/2020 (implement FastSimulator)

In this notebook, I try to reproduce what Gabriele has done. Spectroimaging on the galaxy center, compare the reconstruction to the input sky pixel by pixel

In [ ]:
%matplotlib inline

import os
import sys
import glob
from importlib import reload
import gc
# Specific science modules
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import pickle 
import astropy.io as fits

# Specific qubic modules
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic import QubicSkySim as qss
from qubic.polyacquisition import compute_freq
from qubic import ReadMC as rmc
from qubic import create_acquisition_operator_REC
import ForegroundsSED as fsed
import qubic.AnalysisMC as amc

from pysimulators import FitsArray
import time
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from scipy.optimize import curve_fit
import scipy.constants
from qubic import mcmc
import qubic.AnalysisMC as amc

plt.rc('text',usetex=False)
plt.rc('font', size=16)

from mpl_toolkits.mplot3d import Axes3D

Implementing FastSimulator

In [ ]:
# Dictionary saved during the simulation
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])
#dictfilename = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config)
dictfilename = global_dir + '/dicts/spectroimaging_article.dict'
dFI = qubic.qubicdict.qubicDict()
dFI.read_from_file(dictfilename)
dFI['nf_recon'] = 5
dFI['nf_sub'] = 5
dFI['photon_noise'] = True
dFI['effective_duration'] = 1
dTD = dFI.copy()
dTD['config'] = 'TD'
#dTD['detector_nep']=2.05e-16
dTD['npointings'] = 10000
# Coordinates of the zone observed in the sky
center = np.array([0,0]) #galaxy center
dTD['RA_center'],dTD['DEC_center'] = qubic.gal2equ(center[0],center[1])
dFI['RA_center'],dFI['DEC_center'] = dTD['RA_center'],dTD['DEC_center'] 

regions = ["GalCen_Field"]
bands = ["150"]
centers = [center]
dictionaries = [dTD]

In [ ]:
# NEW (19 Feb 2021)
reload(fsed)
covnameTD = [global_dir+'/doc/FastSimulator/Data/DataFastSimulator_TD150G_coverage.fits',]
coveragesTD = fsed.coverage(dictionaries, regions, bands, filename = covnameTD)
covnameFI = [global_dir+'/doc/FastSimulator/Data/DataFastSimulator_FI150G_coverage.fits',]
coveragesFI = fsed.coverage(dictionaries, regions, bands, filename = covnameFI)

In [ ]:
#NEW
reload(fsed)
fsed._plot_covs(regions, bands, coveragesTD, center, config = "TD")
fsed._plot_covs(regions, bands, coveragesFI, center, config = "FI")

##### QubicSkySim instanciation
seed = None
sky_config_220 = {'dust': 'd1'}
d220=d.copy()
d220['filter_nu']=220*1e9
d220['nf_recon']=5
Qubic_sky_220 = qss.Qubic_sky(sky_config_220, d220)
auxmaps220, noiseless220, _, _ = Qubic_sky_220.get_partial_sky_maps_withnoise(spatial_noise=False, 
                                                                              coverage=coverage)

In [ ]:
# NEW (16 Feb 2021)

reload(fsed)
sky_conf_dust = {'dust': 'd1'}
sky_conf_synch = {'synch': 's0'}
sky_conf_all = {'dust': 'd1', 'synch': 's0'}
fground_maps_dust = fsed.foreground_signal(dictionaries, sky_conf_dust)
fground_maps_synch = fsed.foreground_signal(dictionaries, sky_conf_synch)
fground_maps = fsed.foreground_signal(dictionaries, sky_conf_all)

In [ ]:
# NEW (19 Feb 2021)
reload(fsed)
nreals = 100
#noiseTD = fsed.noise_qss(dictionaries, sky_conf_all, coveragesTD, nreals, verbose = False) 
noiseFI = fsed.noise_qss([dFI], sky_conf_all, coveragesFI, nreals, verbose = False) 

In [ ]:
# NEW (16 Feb 2021)

reload(fsed)
noiseTD = 31 * noiseFI
#for ic, idict in enumerate(dictionaries):
#    noisy_frgrounds = np.zeros(np.shape(noiseFI)[1:])
#    for j in range(nreals):
#        noisy_frgrounds[j, ...] = noiseTD[ic, j, ...] + fground_maps[ic]
#    maps = np.array([np.mean(noisy_frgrounds, axis = 0)])
#    stdmaps = np.array([np.std(noisy_frgrounds, axis = 0)])


### Look at the maps... okk..

In [ ]:
# NEW (19 Feb 2021)
reload(fsed)
nf_recon = dictionaries[0]['nf_recon']
maskmaps_dust, _ = fsed._mask_maps(fground_maps_dust, coveragesTD, nf_recon)
maskmaps_synch, _ = fsed._mask_maps(fground_maps_synch, coveragesTD, nf_recon)
maskmaps_all, _ = fsed._mask_maps(fground_maps, coveragesTD, nf_recon)
#maskmaps, mapregion = fsed._mask_maps(maps, coveragesTD, nf_recon)

In [ ]:
# NEW (17 Feb 2021)
reload(fsed)

_, nus150, nus_out150, _, _, _ = qubic.compute_freq(dictionaries[0]['filter_nu'] / 1e9,  
                            dictionaries[0]['nf_recon'],
                            dictionaries[0]['filter_relative_bandwidth'])

ix = 0

MapsArray = [maskmaps_dust[ix], maskmaps_synch[ix], maskmaps_all[ix]]
fsed._plot_exampleSED(dictionaries[ix], centers[ix], nus_out150, MapsArray, mapsarray = True, 
                     DeltaTheta = 1, DeltaPhi = 1, intensity = False, covmap = coveragesTD)

### Compute the rms in different regions

In [ ]:
#maskmaps_dust, maskmaps_synch, maskmaps_all
#print(coveragesTD[0], )
#cov_lim = 1e3
#cov_lims = 1e1
add_dust = np.zeros((2,2,5))
add_synch = np.zeros((2,2,5))
add_all = np.zeros((2,2,5))
stk = 1

for stk in [0,1]:
    if stk == 0:
        lims = np.array([[1.7,1e2],[1.25e1,3e3]])
    else:
        lims = np.array([[-0.35,-8],[0.48,9]])

    for j, limlim in enumerate(lims):
        cov_lims = limlim[0]
        cov_lim = limlim[1]
        fig , ax = plt.subplots(nrows = 3, ncols = 5, figsize = (14,10))
        for fr in range(5):
            if stk == 0:
                mmm = np.copy(maskmaps_dust[0,fr, :, stk])
                mmsk = maskmaps_dust[0,fr, : ,stk] > cov_lim
                mmm[~mmsk] = hp.UNSEEN

                mmm_s = np.copy(maskmaps_synch[0,fr, :, stk])
                mmsk_s = maskmaps_synch[0,fr, : ,stk] > cov_lims
                mmm_s[~mmsk_s] = hp.UNSEEN

                mmm_a = np.copy(maskmaps_all[0,fr, :, stk])
                mmsk_a = maskmaps_all[0,fr, : ,stk] > cov_lim
                mmm_a[~mmsk_a] = hp.UNSEEN

            else:
                mmm1, mmm2 = np.copy(maskmaps_dust[0,fr, :, stk]), np.copy(maskmaps_dust[0,fr, :, stk+1])
                mmm = np.sqrt(mmm1**2 + mmm2**2)
                mmsk1, mmsk2 = maskmaps_dust[0,fr, : ,stk] > cov_lim, maskmaps_dust[0,fr, : ,stk+1] > cov_lim
                mmsk = mmsk1 * mmsk2
                mmm[~mmsk] = hp.UNSEEN
                #print(np.min(mmm[mmsk]), np.sum(coveragesTD[0] > 0)) 
                mmm_s1, mmm_s2 = np.copy(maskmaps_synch[0,fr, :, stk]), np.copy(maskmaps_synch[0,fr, :, stk+1])
                mmm_s = np.sqrt(mmm_s1**2 + mmm_s2**2)
                mmsk_s1, mmsk_s2 = maskmaps_synch[0,fr, : ,stk] > cov_lims, maskmaps_synch[0,fr, : ,stk+1] > cov_lims
                mmsk_s = mmsk_s1 * mmsk_s2
                mmm_s[~mmsk_s] = hp.UNSEEN
                #print(np.min(mmm_s[mmsk_s]))
                mmm_a1, mmm_a2 = np.copy(maskmaps_all[0,fr, :, stk]), np.copy(maskmaps_all[0,fr, :, stk+1])
                mmm_a = np.sqrt(mmm_a1**2 + mmm_a2**2)
                mmsk_a1, mmsk_a2 = maskmaps_all[0,fr, : ,stk] > cov_lim, maskmaps_all[0,fr, : ,stk+1] > cov_lim
                mmsk_a = mmsk_a1 * mmsk_a2
                mmm_a[~mmsk_a] = hp.UNSEEN

            ax[0,fr].cla()
            plt.axes(ax[0,fr])
            hp.gnomview(mmm, reso = 14, rot = centers[0], 
                        title = "Polvo galáctico" if fr == 2 else " ",
                        max = 6e3 if stk == 0 else 35, hold = True)
            add_dust[stk,j] =np.sqrt(np.mean(mmm[mmsk]**2)) 
            if fr == 2: print("Region coverage (dust) {}".format(sum(mmsk)/25800 ))
            #mmm_s = np.copy(maskmaps_synch[0,fr, :, stk])
            #mmsk_s = maskmaps_synch[0,fr, : ,stk] > cov_lims
            #mmm_s[~mmsk_s] = hp.UNSEEN
            ax[1,fr].cla()
            plt.axes(ax[1,fr])
            hp.gnomview(mmm_s, reso = 14, rot = centers[0], 
                        title = "Sincrotrón" if fr == 2 else " ", 
                        max = 30 if stk == 0 else 9, hold = True)
            add_synch[stk,j] = np.sqrt(np.mean(mmm_s[mmsk_s]**2)) 
            if fr == 2: print("Region coverage (synch) {}".format(sum(mmsk_s)/25800 ))

            #mmm_a = np.copy(maskmaps_all[0,fr, :, stk])
            #mmsk_a = maskmaps_all[0,fr, : ,stk] > cov_lim
            #mmm_a[~mmsk_a] = hp.UNSEEN
            plt.axes(ax[2,fr])
            hp.gnomview(mmm_a, reso = 14, rot = centers[0], 
                        title = "Polvo galáctico + sincrotrón" if fr == 2 else " ",
                        max = 6e3 if stk == 0 else 35, hold = True)
            add_all[stk,j] = np.sqrt(np.mean(mmm_a[mmsk_a]**2)) 
            if fr == 2: print("Region coverage (dust+synch) {}".format(sum(mmsk_a)/25800 ))
        plt.tight_layout()
        #plt.savefig("RegionSky_SED{}".format(j))

In [ ]:
for stk in [0,1]:
    plt.fill_between(nus_out150, y1 = add_dust[stk,0], y2 = add_dust[stk,1],
                     color = 'b' if stk == 0 else 'g', alpha = 0.6, 
                     linestyle = "--", lw = 4,
                    label = "Temperatura" if stk == 0 else "Polarización")
    #if stk == 0:
    #    plt.text(nus_out150[0], add_dust[0,0]+0.2, "$f_{region} \sim 0.8$", c = "k")
    #    plt.text(nus_out150[0], add_dust[1,0]+1e3, "$f_{region} \sim 0.05$", c = "k")
    #else:
    #    plt.text(nus_out150[0], add_dust[0,0]+0.2, "$f_{region} \sim 0.8$", c = "k")
    #    plt.text(nus_out150[0], add_dust[1,0]+1, "$f_{region} \sim 0.05$", c = "k")
    plt.fill_between(nus_out150, y1 = add_synch[stk,0], y2 = add_synch[stk,1],
                     color = 'b' if stk == 0 else 'g', alpha = 0.6, 
                     linestyle = "--", lw = 4)

    #if stk == 0: 
    #    plt.text(nus_out150[0], add_synch[0,0]+1, "$f_{region} \sim 0.8$", c = "k")
    #    plt.text(nus_out150[0], add_synch[1,0]+3, "$f_{region} \sim 0.05$", c = "k")
    #else:
    #    plt.text(nus_out150[0], add_synch[0,0], "$f_{region} \sim 0.8$", c = "k")
    #    plt.text(nus_out150[0], add_synch[1,0]+0.2, "$f_{region} \sim 0.05$", c = "k")
    plt.fill_between(nus_out150, y1 = add_all[stk,0], y2 = add_all[stk,1],
                       color = 'r', alpha = 0.8)
    plt.ylabel(r"$rms(\nu)~[\mu K_{\rm CMB}]$")# if stk == 0 else r"$P(\nu)~[\mu K_{\rm CMB}]$", fontsize = 18)
    plt.xlabel(r"$\nu [GHz]$", fontsize = 18)
    plt.yscale("log")
    #if stk == 0:
    #    plt.ylim(-0.2,3e4)
    #else:
    #    plt.ylim(-0.2,1e2)
    plt.ylim(3e-1,5e4)
    plt.legend(loc = "upper center", ncol = 2)
plt.tight_layout()
#plt.savefig("rms_Comp-Dust_synch")

## Udgrading maps

In [ ]:
# NEW (19 Feb 2021)
reload(fsed)
nside_new = 64
nf_recon = dictionaries[0]['nf_recon']

_, _, fgr_map_dust_ud, _ = fsed.udgrade_maps(fground_maps_dust, noiseTD, nside_new, 
                                            nf_recon, nreals, return_with_noise = True)
_, _, fgr_map_synch_ud, _ = fsed.udgrade_maps(fground_maps_synch, noiseTD, nside_new, 
                                            nf_recon, nreals, return_with_noise = True)
maps_ud, std_ud, fgr_map_ud, noise_ud_i = fsed.udgrade_maps(fground_maps, noiseTD, nside_new, 
                                            nf_recon, nreals, return_with_noise = True)

_, covmask = fsed._mask_maps(fground_maps, coveragesTD, nf_recon)
cov_ud = hp.ud_grade(covmask, nside_new)

print("cov_ud shape", np.shape(cov_ud))
Cp = []
Cp_prime = []
for br in range(len(dictionaries)):
    Cp = amc.get_Cp(noise_ud_i[br][:, :, cov_ud[br], :], verbose = False, doplot = False)
    Cp_prime.append(amc.Cp2Cp_prime_viaCorr(Cp, verbose = False)[1])

In [ ]:
bandreg = ["GalCen_Field150TD", ]
for j, idict in enumerate(dictionaries):
    FitsArray(Cp_prime[j]).save("DataSED/2022/SED_CovarMatrix_config{}_nside{}_nreals{}_{}.fits".format(bandreg[j],
                                                                                            nside_new,
                                                                                            nreals,
                                                                              "150"))

In [ ]:
PixPix = lambda p: hp.ang2pix(nside_new, hp.pix2ang(dTD['nside'], p)[0], hp.pix2ang(dTD['nside'], p)[1] )

In [ ]:
Cp_prime[0].shape

In [ ]:
plt.figure()
plt.imshow(Cp_prime[0][:, :, 0, 0])
plt.colorbar()

In [ ]:
# NEW (17 feb 2021)
for idict in range(len(dictionaries)):
    for iNU in range(nf_recon):
        maps_ud[idict, iNU,~cov_ud[idict],:] = hp.UNSEEN

### FIT

In [ ]:
# NEW (19 Feb 2021)
#
#   Choose pixel for regions
#

pixG = [hp.ang2pix(dictionaries[0]['nside'], 
                   np.pi / 2 - np.deg2rad(centers[0][1] + 8),  np.deg2rad(centers[0][0] - 0 ) ), ]

pixG_ud = PixPix(pixG[0])
# Where the sky pixel is in the reduce format (pixels seen array and not full map)
pixG_red = np.where(np.where(cov_ud[0] == True)[0] == pixG_ud)[0][0]

plt.figure(figsize = (10,4))
hp.gnomview(maps_ud[0,-1,:,0], reso = 15,#hold = True, 
            notext = False, title = 'G patch ', sub = (121),
            max = 0.4*np.max(maps_ud[0,-1,:,0]), 
            unit = r'$\mu$K',
            rot = centers[0])
hp.projscatter(hp.pix2ang(nside_new, pixG_ud), marker = '*', color = 'r', s = 200)
hp.graticule(dpar = 10, dmer = 20, alpha = 0.6)

In [ ]:
# NEW (19 Feb 2021)

reload(fsed)
nus_out = [nus_out150, ]
pixs_ud = [pixG_ud, ]
pixs_red = [pixG_red, ]
nus_edge = [nus_edge150, ]

study = "dust"

if study == "dust":
    FuncModel = fsed.ThermDust_Planck545
    p0 = np.array([1e5,3])
elif study == "synch":
    FuncModel = fsed.Synchrotron_storja
    #p0 = np.array([1e1,20,-3]) #Planch
    p0 = np.array([1e1,3])
elif study == "dust+synch":
    FuncModel = fsed.DustSynch_model
    p0 = np.array([1e6, 3, 1e4, 3])
    
    
Chi2Model = None#"Chi2Implement"

Imvals, Isvals, xarr, _ = fsed.foregrounds_run_mcmc(dictionaries, fgr_map_ud, Cp_prime, FuncModel,
                                                nus_out, nus_edge, pixs_ud, pixs_red = pixs_red, 
                                                chi2 = Chi2Model, samples = 5000, verbose = True, 
                                                 initP0 = p0)

In [ ]:
xSED = [nus_out150, ]

if study == "dust":
    FuncPoint = fsed.ThermDust_Planck353_pointer
    #FuncPoint = fsed.ThermDust_Planck545_pointer
    initGuess = np.array([1e4, 3])
    
elif study == "synch":
    FuncPoint = fsed.Synchrotron_storja_pointer
    #FuncPoint = fsed.Synchrotron_Planck_pointer

elif study == "dust+synch":
    FuncPoint = fsed.DustSynch_model_pointer
    initGuess = np.array([1e3, 3, 1e8, 3])
    
ySED_fit, Pmean, Perr = fsed.make_fit_SED(xSED, xarr, Imvals, Isvals,
                                          FuncPoint, fgr_map_ud, pixs_ud, nf_recon, 
                                          initP0 = initGuess, 
                                          maxfev = 15000)

In [ ]:
#plt.hist(flat_samples[0,:,0], bins = 20, histtype="step")
savefigs = True

_, nus220, nus_out220, _, _, _ = qubic.compute_freq(220,  
                            dictionaries[0]['nf_recon'],
                            dictionaries[0]['filter_relative_bandwidth'])

# NEW (17 feb 2021)
RESO = 15
capsize = 3
plt.rc('font', size = 14)

fig,ax = plt.subplots(nrows = 1, ncols = 2,figsize = (12,6), gridspec_kw = {'wspace': 0.4})
ax = ax.ravel()
plt.subplots_adjust(wspace = 0.1)
# Plotting
# Dust galactic center
t0, = ax[0].plot(nus_out150, fgr_map_dust_ud[0,:,pixs_ud[0],0], ls = '', 
           marker = 'd', color = 'g',alpha = 0.5, label = 'Dust')
#Synch galactic center
t1, = ax[0].plot(nus_out150, fgr_map_synch_ud[0,:,pixs_ud[0],0], ls = '', 
           marker = 's', color = 'g', alpha = 0.5, label = 'Synchrotron')
#Two components
p1, = ax[0].plot(nus_out150, fgr_map_ud[0, :, pixs_ud[0], 0], 'ro', label = 'Input sky')

e1 = ax[0].fill_between(xarr[0,:], y1 = ySED_fit[0,:,0] - Isvals[0, :, 0], 
                                y2 = ySED_fit[0, :, 0] + Isvals[0, :, 0], 
                 color = 'r', alpha = 0.3, label = '68% C.L.')

# Settings
greyscale = 0.1
xlim = ax[0].get_xlim()
ylim = ax[0].get_ylim()
ax[0].axvspan(nus150[-1], nus220[0],color='k',alpha = greyscale)
ax[0].axvspan(xlim[0], nus150[0], color = 'k', alpha = greyscale)
ax[0].axvspan(nus220[-1], xlim[-1], color = 'k', alpha = greyscale)

ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)

#l = ax[0].legend([(t0,t1), (p1, p2), (e1, e2)], ["Dust - Synch", 'Full ', '68% C.L.'], numpoints=1, 
#                 loc = 2, fontsize = 12,
#               handler_map={tuple: HandlerTuple(ndivide=None)})

ax[0].grid()
ax[0].set_title('GC patch - {} year'.format(dictionaries[0]['effective_duration']),fontsize=16)
ax[0].set_ylabel(r'$I(\nu)$ [$\mu$K]',fontsize=16)
ax[0].set_xlabel(r'$\nu$[GHz]',fontsize=16)
ax[0].legend()
# Displaying maps
ax[1].cla()
plt.axes(ax[1])
hp.gnomview(maps_ud[0, -1, :, 0], reso = 15,hold = True, 
            notext = True, title = ' ',
            min = 0,
            max = 0.4*np.max(maps_ud[0, -1, :, 0]), 
            unit = r'$\mu$K',
            rot = centers[0])
hp.projscatter(hp.pix2ang(nside_new, pixs_ud[0]), marker = '*', color = 'r', s = 180)
dpar = 10
dmer = 20
#Watch out, the names are wrong (change it)
mer_coordsG = [centers[0][0] - dmer, centers[0][0], centers[0][0] + dmer]
long_coordsG = [centers[0][1] - 2*dpar, centers[0][1] - dpar, 
                centers[0][1], centers[0][1] + dpar, centers[0][1] + 2 * dpar]
#paralels
for ilong in long_coordsG:
    plt.text(np.deg2rad(mer_coordsG[0] - 12), 1.1*np.deg2rad(ilong), 
             r'{}$\degree$'.format(ilong))
#meridians
for imer in mer_coordsG:
    if imer < 0:
        jmer = imer + 360
        ip, dp = divmod(jmer/15,1)
    else:
        ip, dp = divmod(imer/15,1)
    if imer == 0:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(int(ip) ))
    else:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(imer))
             #r'{}h{}m'.format(int(ip), int(round(dp*60))))
hp.projtext(mer_coordsG[1] + 2, long_coordsG[0] - 6, '$l$',  color = 'k', lonlat=True)
hp.projtext(mer_coordsG[2] + 12.5, long_coordsG[2] - 1, '$b$', rotation = 90, color = 'k', lonlat=True)

hp.graticule(dpar = dpar, dmer = dmer, alpha = 0.6, verbose = False)

plt.tight_layout()
if savefigs:
    plt.savefig('Figs-TD-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixG{}_Intensity_grat.svg'.format(nside_new,
                                                                            FuncModel.__name__,
                                                                            nf_recon,nside_new,
                                                           pixG_ud), 
            format = 'svg', bbox_inches='tight')
    plt.savefig('Figs-TD-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixG{}_Intensity_grat.pdf'.format(nside_new,
                                                                                FuncModel.__name__,
                                                                               nf_recon,nside_new,
                                                           pixG_ud), 
            format = 'pdf', bbox_inches='tight')
    plt.savefig('Figs-TD-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixG{}_Intensity'.format(nside_new,
                                                                        FuncModel.__name__, 
                                                                           nf_recon,
                                                                           nside_new, pixG_ud),
           bbox_inches='tight')
else:
    plt.show()


## Polarization


In [ ]:
#plt.hist(flat_samples[0,:,0], bins = 20, histtype="step")
savefigs = True

_, nus220, nus_out220, _, _, _ = qubic.compute_freq(220,  
                            dictionaries[0]['nf_recon'],
                            dictionaries[0]['filter_relative_bandwidth'])

# NEW (17 feb 2021)
RESO = 15
capsize = 3
plt.rc('font', size = 14)

fig,ax = plt.subplots(nrows = 1, ncols = 2,figsize = (12,6), gridspec_kw = {'wspace': 0.4})
ax = ax.ravel()
plt.subplots_adjust(wspace = 0.1)
# Plotting
# Dust galactic center
t0, = ax[0].plot(nus_out150, np.sqrt(fgr_map_dust_ud[0,:,pixs_ud[0],1] ** 2 + \
                 fgr_map_dust_ud[0,:,pixs_ud[0],2] **2), ls = '', 
           marker = 'd', color = 'g',alpha = 0.5, label = 'Dust')
#Synch galactic center
t1, = ax[0].plot(nus_out150, np.sqrt(fgr_map_synch_ud[0,:,pixs_ud[0],1] ** 2 + \
                                    fgr_map_synch_ud[0,:,pixs_ud[0],2] ** 2), ls = '', 
           marker = 's', color = 'g', alpha = 0.5, label = 'Synchrotron')
#Two components
p1, = ax[0].plot(nus_out150, np.sqrt(fgr_map_ud[0,:,pixs_ud[0],1] ** 2 + \
                                     fgr_map_ud[0,:,pixs_ud[0],2] ** 2),
                 'ro', label = 'Input sky')

e1 = ax[0].fill_between(xarr[0,:], y1 = ySED_fit[0, :, 1] - Perr[0], 
                        y2 = ySED_fit[0, :, 1] + Perr[0], 
                 color = 'r', alpha = 0.3, label = '68% C.L.')

# Settings
greyscale = 0.1
xlim = ax[0].get_xlim()
ylim = ax[0].get_ylim()
ax[0].axvspan(nus150[-1], nus220[0],color='k',alpha = greyscale)
ax[0].axvspan(xlim[0], nus150[0], color = 'k', alpha = greyscale)
ax[0].axvspan(nus220[-1], xlim[-1], color = 'k', alpha = greyscale)

ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)

#l = ax[0].legend([(t0,t1), (p1, p2), (e1, e2)], ["Dust - Synch", 'Full ', '68% C.L.'], numpoints=1, 
#                 loc = 2, fontsize = 12,
#               handler_map={tuple: HandlerTuple(ndivide=None)})
ax[0].legend()
ax[0].set_title('TD - {} year'.format(dictionaries[0]['effective_duration']), fontsize = 14)
ax[0].set_ylabel(r'$P(\nu)~[\mu$K]', fontsize = 14)
ax[0].set_xlabel(r'$\nu~[GHz]$', fontsize = 14)
ax[0].legend(loc = 2, fontsize = 12)
ax[0].grid()

# Displaying maps
auxmapG = np.sqrt(maps_ud[0, 0, :, 1] ** 2 + maps_ud[0, 0, :, 2] ** 2)
auxmapG[~cov_ud[0]] = hp.UNSEEN
ax[1].cla()
plt.axes(ax[1])
hp.gnomview(auxmapG, reso = 15,hold = True, 
            notext = True, title = ' ',
            min = 0,
            max = 0.9*np.max(auxmapG), 
            unit = r'$\mu$K',
            rot = centers[0])
hp.projscatter(hp.pix2ang(nside_new, pixs_ud[0]), marker = '*', color = 'r', s = 180)
dpar = 10
dmer = 20
#Watch out, the names are wrong (change it)
mer_coordsG = [centers[0][0] - dmer, centers[0][0], centers[0][0] + dmer]
long_coordsG = [centers[0][1] - 2*dpar, centers[0][1] - dpar, 
                centers[0][1], centers[0][1] + dpar, centers[0][1] + 2 * dpar]
#paralels
for ilong in long_coordsG:
    plt.text(np.deg2rad(mer_coordsG[0] - 12), 1.1*np.deg2rad(ilong), 
             r'{}$\degree$'.format(ilong))
#meridians
for imer in mer_coordsG:
    if imer < 0:
        jmer = imer + 360
        ip, dp = divmod(jmer/15,1)
    else:
        ip, dp = divmod(imer/15,1)
    if imer == 0:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(int(ip) ))
    else:
        plt.text(-np.deg2rad(imer + 3), np.deg2rad(long_coordsG[-1] + 6), 
             r'{}$\degree$'.format(imer))
             #r'{}h{}m'.format(int(ip), int(round(dp*60))))
hp.projtext(mer_coordsG[1] + 2, long_coordsG[0] - 6, '$l$',  color = 'k', lonlat=True)
hp.projtext(mer_coordsG[2] + 12.5, long_coordsG[2] - 1, '$b$', rotation = 90, color = 'k', lonlat=True)

hp.graticule(dpar = dpar, dmer = dmer, alpha = 0.6, verbose = False)

plt.tight_layout()
if savefigs:
    plt.savefig('Figs-TD-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixG{}_Polarization_grat.svg'.format(nside_new,
                                                                            FuncModel.__name__,
                                                                            nf_recon,nside_new,
                                                            pixG_ud), 
            format = 'svg', bbox_inches='tight')
    plt.savefig('Figs-TD-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixG{}_Polarization_grat.pdf'.format(nside_new,
                                                                                FuncModel.__name__,
                                                                               nf_recon,nside_new,
                                                            pixG_ud), 
            format = 'pdf', bbox_inches='tight')
    plt.savefig('Figs-TD-SED/March2021/NSIDE{}/{}_nrec{}_nside{}_pixG{}_Polarization'.format(nside_new,
                                                                        FuncModel.__name__, 
                                                                           nf_recon,
                                                                           nside_new, pixG_ud),
           bbox_inches='tight')
else:
    plt.show()
